In [ ]:
import numpy as np
import cv2
import pandas as pd
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Add,Concatenate,concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  GlobalAveragePooling2D,Dropout,Conv2D,MaxPool2D,UpSampling2D,LeakyReLU,AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
import keras.backend as K
from keras.layers import Layer

In [ ]:
train_load= np.load("DATA/B_train.npy")

In [ ]:
test_load =np.load("DATA/B_test.npy")

In [ ]:
Y_train=np.load("DATA/Y_train.npy")

In [ ]:
Y_test=np.load("DATA/Y_test.npy")

In [ ]:
Y=[]
for i in range(len(Y_train)):
    Y.append(int(Y_train[i]))

In [ ]:
Y1=[]
for i in range(len(Y_test)):
    Y1.append(int(Y_test[i]))

In [ ]:
FINAL_LABEL=[]
for j in range(len(Y)):
    if Y[j]==1:
        FINAL_LABEL.append([1,0,0,0])
    elif Y[j]==2:
        FINAL_LABEL.append([0,1,0,0])
    elif Y[j]==3:
        FINAL_LABEL.append([0,0,1,0])
    else:
        FINAL_LABEL.append([0,0,0,1])

In [ ]:
FINAL_LABEL_test=[]
for j in range(len(Y1)):
    if Y1[j]==1:
        FINAL_LABEL_test.append([1,0,0,0])
    elif Y1[j]==2:
        FINAL_LABEL_test.append([0,1,0,0])
    elif Y1[j]==3:
        FINAL_LABEL_test.append([0,0,1,0])
    else:
        FINAL_LABEL_test.append([0,0,0,1])

In [ ]:
y1=np.array(FINAL_LABEL)
y2=np.array(FINAL_LABEL_test)

In [ ]:
# @tf.function
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W1=self.add_weight(name='attention_weight1', shape=(128,1),
                               initializer='random_normal', trainable=True)
        self.b1=self.add_weight(name='attention_bias1', shape=(1,),
                               initializer='zeros', trainable=True)
        self.W2=self.add_weight(name='attention_weight2', shape=(128,1),
                               initializer='random_normal', trainable=True)
        self.b2=self.add_weight(name='attention_bias2', shape=(1,),
                               initializer='zeros', trainable=True)
        self.W3=self.add_weight(name='attention_weight3', shape=(128,1),
                               initializer='random_normal', trainable=True)
        self.b3=self.add_weight(name='attention_bias3', shape=(1,),
                               initializer='zeros', trainable=True)

        self.W4=self.add_weight(name='attention_weight4', shape=(3,1),
                               initializer='random_normal', trainable=True)
        self.b4=self.add_weight(name='attention_bias4', shape=(1,),
                               initializer='zeros', trainable=True)
        super(attention, self).build(input_shape)

    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e1 = K.relu(K.dot(x,self.W1)+self.b1)
        e2 = K.relu(K.dot(x,self.W2)+self.b2)
        e3 = K.relu(K.dot(x,self.W3)+self.b3)
        e_head=tf.concat([e1, e2,e3],-1)
        e = K.relu(K.dot(e_head,self.W4)+self.b4)
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context

In [ ]:
def CNN(input_shape):
    X_input = Input(input_shape)
    conv1=Conv2D(32, kernel_size=(3, 3),padding="same",strides=(1,1))(X_input)
    conv1=BatchNormalization()(conv1)
    conv1=LeakyReLU(alpha=0.1)(conv1)



    ## downsampling
    conv2=Conv2D(64, kernel_size=(3, 3),strides=(2,2))(conv1)
    conv2=BatchNormalization()(conv2)
    conv2=LeakyReLU(alpha=0.1)(conv2)



    ## 1st block
    conv3=Conv2D(32, kernel_size=(1, 1),
                     strides=(1,1),padding="same")(conv2)
    conv3=BatchNormalization()(conv3)
    conv3=LeakyReLU(alpha=0.1)(conv3)


    conv4=Conv2D(64, kernel_size=(3, 3),
                     strides=(1,1),padding="same")(conv3)
    conv4=BatchNormalization()(conv4)
    conv4=LeakyReLU(alpha=0.1)(conv4)

    conv5=Add()([conv2,conv4])


    ## DOWNSAMPLING
    conv6=Conv2D(128, kernel_size=(3, 3),strides=(2,2))(conv5)
    conv6=BatchNormalization()(conv6)
    conv6=LeakyReLU(alpha=0.1)(conv6)

    ## 2nd block
    conv7=Conv2D(64, kernel_size=(1, 1),strides=(1,1),padding="same")(conv6)
    conv7=BatchNormalization()(conv7)
    conv7=LeakyReLU(alpha=0.1)(conv7)

    conv8=Conv2D(128, kernel_size=(3, 3),strides=(1,1),padding="same")(conv7)
    conv8=BatchNormalization()(conv8)
    conv8=LeakyReLU(alpha=0.1)(conv8)

    conv9=Add()([conv6,conv8])

    conv10=Conv2D(64, kernel_size=(1, 1),strides=(1,1),padding="same")(conv9)


    ## downsampling
    conv13=Conv2D(512, kernel_size=(3, 3),
                    strides=(2,2))(conv10)
    conv13=BatchNormalization()(conv13)
    conv13=LeakyReLU(alpha=0.1)(conv13)

    ## 3rd block
    conv14=Conv2D(256, kernel_size=(1, 1),
                    strides=(1,1),padding="same")(conv13)
    conv14=BatchNormalization()(conv14)
    conv14=LeakyReLU(alpha=0.1)(conv14)

    conv15=Conv2D(512, kernel_size=(3, 3),
                    strides=(1,1),padding="same")(conv14)
    conv15=BatchNormalization()(conv15)
    conv15=LeakyReLU(alpha=0.1)(conv15)

    conv16=Add()([conv13,conv15])

    conv17=Conv2D(256, kernel_size=(1, 1),
                    strides=(1,1),padding="same")(conv16)
    conv17=BatchNormalization()(conv17)
    conv17=LeakyReLU(alpha=0.1)(conv17)

    conv18=Conv2D(512, kernel_size=(3, 3),
                    strides=(1,1),padding="same")(conv17)
    conv18=BatchNormalization()(conv18)
    conv18=LeakyReLU(alpha=0.1)(conv18)

    conv19=Add()([conv16,conv18])

#     ## downsampling
    conv26=Conv2D(1024, kernel_size=(3, 3),
                    strides=(2,2))(conv19)
    conv26=BatchNormalization()(conv26)
    conv26=LeakyReLU(alpha=0.1)(conv26)
    ## 4th block
    conv27=Conv2D(512, kernel_size=(1, 1),
                    strides=(1,1),padding="same")(conv26)
    conv27=BatchNormalization()(conv27)
    conv27=LeakyReLU(alpha=0.1)(conv27)

    conv28=Conv2D(1024, kernel_size=(3, 3),
                    strides=(1,1),padding="same")(conv27)
    conv28=BatchNormalization()(conv28)
    conv28=LeakyReLU(alpha=0.1)(conv28)

    conv29=Add()([conv26,conv28])

    conv30=Conv2D(512, kernel_size=(1, 1),
                    strides=(1,1),padding="same")(conv29)
    conv30=BatchNormalization()(conv30)
    conv30=LeakyReLU(alpha=0.1)(conv30)

    conv31=Conv2D(1024, kernel_size=(3, 3),
                    strides=(1,1),padding="same")(conv30)
    conv31=BatchNormalization()(conv31)
    conv31=LeakyReLU(alpha=0.1)(conv31)
    conv32=Add()([conv29,conv31])
    #     ## downsampling
    conv33=Conv2D(1024, kernel_size=(3, 3),
                    strides=(2,2))(conv32)
    conv33=BatchNormalization()(conv33)
    conv33=LeakyReLU(alpha=0.1)(conv33)
    ## 4th block
    conv34=Conv2D(512, kernel_size=(1, 1),
                    strides=(1,1),padding="same")(conv33)
    conv34=BatchNormalization()(conv34)
    conv34=LeakyReLU(alpha=0.1)(conv34)

    conv35=Conv2D(1024, kernel_size=(3, 3),
                    strides=(1,1),padding="same")(conv34)
    conv35=BatchNormalization()(conv35)
    conv35=LeakyReLU(alpha=0.1)(conv35)

    conv36=Add()([conv33,conv35])

    conv37=Conv2D(512, kernel_size=(1, 1),
                    strides=(1,1),padding="same")(conv36)
    conv37=BatchNormalization()(conv37)
    conv37=LeakyReLU(alpha=0.1)(conv37)

    conv38=Conv2D(1024, kernel_size=(3, 3),
                    strides=(1,1),padding="same")(conv37)
    conv38=BatchNormalization()(conv38)
    conv38=LeakyReLU(alpha=0.1)(conv38)
    conv39=Add()([conv36,conv38])
    conv100=Flatten()(conv39)
    conv105=Dense(128, activation='ReLU')(conv100)
    conv106=Dense(64, activation='ReLU')(conv105)
    conv107=Dense(32, activation='ReLU')(conv106)
    conv108=Dense(16, activation='ReLU')(conv107)
    prediction = Dense(4, activation='softmax')(conv108)
    model=Model(inputs=X_input, outputs=prediction)
    return model

In [ ]:
input_shape= (224,224,1)
cnn = CNN(input_shape)
cnn.summary()

In [ ]:
cnn.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
from keras.callbacks import ModelCheckpoint
mc = ModelCheckpoint('1model_B_high.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [ ]:
r = cnn.fit(
  train_load,y1,
  validation_data=(test_load,y2),
  epochs=50,batch_size=32,
  callbacks=[mc])

In [ ]:
cnn.save("/content/drive/MyDrive/DATA/1model_B_high.h5")

In [ ]:
base_model = cnn
# Choose a specific layer to extract features from
desired_layer = base_model.get_layer('flatten')
feature_extractor_model = Model(inputs=base_model.input, outputs=desired_layer.output)

In [ ]:
F=[]
for i in tqdm(range(len(train_load))):
    image=train_load[i].reshape([-1,224,224,1])
    features=feature_extractor_model.predict(image)
    F.append(features)

In [ ]:
np.save("/content/drive/MyDrive/DATA/B_features_train_CNN.npy",F)

In [ ]:
train_load= np.load("DATA/B_features_train_CNN.npy")

In [ ]:
test_load =np.load("B_features_test_CNN.npy")

In [ ]:
def attentions(input_shape):
    X_input = Input(input_shape)
    A1 = attention(input_shape=input_shape)(X_input)
    prediction = Dense(4, activation='softmax')(A1)
    model=Model(inputs=X_input, outputs=prediction)
    return model

In [ ]:
input_shape=(128,)

In [ ]:
att = attentions(input_shape)
att.summary()

In [ ]:
att.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
r1 = att.fit(
  train,y1,
  validation_data=(test,y2),
  epochs=500,batch_size=64)

In [ ]:
base_model = att
# Choose a specific layer to extract features from
desired_layer = base_model.get_layer('attention')
feature_extractor_model = Model(inputs=base_model.input, outputs=desired_layer.output)

In [ ]:
Features=[]
for i in tqdm(range(len(train))):
    x=train[i].reshape([-1,128])
    m=feature_extractor_model.predict(x)
    Features.append(m)

In [ ]:
np.save("/content/drive/MyDrive/DATA/R_features_train_CNN_attention.npy",Features)